## Auto-fitting SARIMA Model For All Wards

### 1. Load and Preprocess Burglary Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima, ADFTest
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data and aggregate to ward-monthly counts
burglary = pd.read_csv('../data/residential_burglary.csv')

# Combine 'Year' and 'Month' into a single datetime column
burglary ["Month"] = pd.to_datetime(burglary ["Year"].astype(str) + "-" + burglary ["Month"].astype(str), format="%Y-%m")

burglary_monthly = burglary.groupby(["Ward Code", "Month"]).size().reset_index(name="Count")

# Extract the year from the 'Month' column for merging with the population data
burglary_monthly['Year'] = burglary_monthly['Month'].dt.year

print("Raw data overview:")
print(f"Total wards: {burglary_monthly['Ward Code'].nunique()}")
print(f"Time range: {burglary_monthly['Month'].min()} to {burglary_monthly['Month'].max()}")

burglary_monthly


Raw data overview:
Total wards: 679
Time range: 2013-12-01 00:00:00 to 2025-02-01 00:00:00


,Ward Code,Month,Count,Year
0,E05009317,2013-12-01,8,2013
1,E05009317,2014-01-01,23,2014
2,E05009317,2014-02-01,12,2014
3,E05009317,2014-03-01,11,2014
4,E05009317,2014-04-01,21,2014
...,...,...,...,...
89633,E05014119,2024-10-01,1,2024
89634,E05014119,2024-11-01,2,2024
89635,E05014119,2024-12-01,4,2024
89636,E05014119,2025-01-01,3,2025


### 2. Define Oscillation Capture Score for Evaluating the Fitting of SARIMA Models

In [2]:
models_order_df = pd.read_csv('../data/density_models_orders.csv')
models_order_df.rename(columns={'Ward code': 'Ward Code'}, inplace=True)


def oscillation_capture_score(actual, predicted):
    """Calculate a composite score that detects oscillation pattern matching"""
    n = len(actual)
    if n < 3:
        return 0.0  # Not enough data

    # 1. Detrended Variability Ratio (DVR)
    var_actual = np.var(actual)
    var_pred = np.var(predicted)

    # Handle near-zero variance cases
    if var_actual < 1e-10 and var_pred < 1e-10:
        dvr = 1.0
    elif var_actual < 1e-10:
        dvr = 0.0
    else:
        dvr = min(var_pred / var_actual, 2.0)  # Cap at 2.0

    # 3. Smoothness Penalty (SP)
    pred_diff = np.diff(predicted)
    sp = 1 - np.exp(-np.mean(np.abs(pred_diff)) / max(1, np.mean(np.abs(np.diff(actual)))))

    # Combine components
    ocs = 0.3 * dvr + 0.7 * sp
    return ocs, dvr, sp

models_order_df

,Ward Code,Order,Seasonal order
0,E05014058,"(1, 0, 0)","(2, 1, 1, 12)"
1,E05014071,"(0, 0, 1)","(0, 1, 1, 12)"
2,E05014059,"(0, 0, 0)","(1, 1, 2, 12)"
3,E05014069,"(0, 0, 0)","(2, 1, 0, 12)"
4,E05014063,"(0, 0, 0)","(3, 1, 0, 12)"
...,...,...,...
673,E05013922,"(1, 0, 1)","(3, 1, 0, 12)"
674,E05013759,"(1, 0, 1)","(2, 1, 1, 12)"
675,E05009326,"(0, 0, 0)","(2, 1, 1, 12)"
676,E05013934,"(0, 0, 0)","(0, 1, 1, 12)"


### 3. SARIMA Forecasting for All Wards

In [3]:
import warnings

warnings.filterwarnings("ignore")

def analyze_ward(ward_code, ward_df, test_months=26):
    """Analyze a single ward's SARIMA performance"""
    try:
        ward_df = ward_df[['Month', 'Count']].set_index('Month').asfreq('MS')

        if ward_df.isnull().values.any():
            print(f"{ward_code} has NaN values")
            if ward_df.isnull().sum().sum() > 10:
                print(f"{ward_code} has more than 10 NaN values, aborted")
                return None
            ward_df.fillna(0, inplace=True)

        # Filter wards with sufficient history
        if len(ward_df) < test_months * 2:
            print(f"{ward_code} has insufficient data")
            print(ward_df)
            return None

        # Split data
        train = ward_df.iloc[:-test_months]
        test = ward_df.iloc[-test_months:]

        # Use per-computed model orders from previous run to save time
        # model = SARIMAX(train["Count"],
        #         order=ast.literal_eval(model_order[["Order"]].values[0][0]),  # Use the correct normal order
        #        seasonal_order=ast.literal_eval(model_order[["Seasonal order"]].values[0][0]))

        # Auto-fit SARIMA
        fitted_model = auto_arima(train['Count'], start_p=0, d=0, start_q=0, max_p=3, max_d=3, max_q=3,
                                 start_P=0, D=1, start_Q=0, max_P=3, max_Q=3, m=12,
                                 seasonal=True, error_action='ignore', suppress_warnings=True, stepwise=True,
                                 random_state=2025, n_fits=10,
                                 stationary=ADFTest(alpha=0.05).should_diff(train['Count'])[1])

        model = SARIMAX(train['Count'],
                       order=fitted_model.order,  # Use the correct normal order
                       seasonal_order=(fitted_model.order[0], fitted_model.order[1], fitted_model.order[2], 12))

        results = model.fit(disp=False)

        # Generate predictions
        preds = results.get_forecast(steps=test_months).predicted_mean

        # Calculate residuals
        residuals = np.subtract(test['Count'].to_numpy(), preds)

        print(f"{ward_code} Completed")

        return {
            'ward_code': ward_code,
            'train_size': len(train),
            'test_size': len(test),
            'residuals': residuals,
            'predicted': preds,
            "fitting_score": oscillation_capture_score(test["Count"].values, preds.values),
            'actual': test['Count'].values,
            'model_order': str(model.order),
            'model_seasonal_order': str(model.seasonal_order)
        }
    except Exception as e:
        print(f"Error processing {ward_df['Ward Code'].iloc[0]}: {str(e)}")
        return None

# Process all wards
results = []
# i=0
for ward_code, group in burglary_monthly.groupby('Ward Code'):
    # if i > 10:
    #     break
    analysis = analyze_ward(ward_code, group)
    if analysis:
        results.append(analysis)
        # i += 1


# with mp.Pool(processes=mp.cpu_count()) as pool:
#     args = [(ward_code, group) for ward_code, group in burglary_monthly.groupby('Ward Code')]
#     results = pool.map(analyze_ward, args)

# Remove None results
results = [res for res in results if res is not None]

print(f"\nSuccessfully processed {len(results)} wards with sufficient data")

warnings.filterwarnings("default")


E05009317 Completed
E05009318 Completed
E05009319 Completed
E05009320 Completed
E05009321 has NaN values
E05009321 Completed
E05009322 Completed
E05009323 Completed
E05009324 has NaN values
E05009324 Completed
E05009325 Completed
E05009326 has NaN values
E05009326 has more than 10 NaN values, aborted
E05009327 Completed
E05009328 has NaN values
E05009328 has more than 10 NaN values, aborted
E05009329 Completed
E05009330 Completed
E05009331 Completed
E05009332 Completed
E05009333 Completed
E05009334 Completed
E05009335 Completed
E05009336 Completed
E05009367 has NaN values
E05009367 Completed
E05009368 has NaN values
E05009368 Completed
E05009369 Completed
E05009370 Completed
E05009371 Completed
E05009372 Completed
E05009373 Completed
E05009374 Completed
E05009375 Completed
E05009376 Completed
E05009377 Completed
E05009378 Completed
E05009379 Completed
E05009380 Completed
E05009381 Completed
E05009382 Completed
E05009383 Completed
E05009384 has NaN values
E05009384 Completed
E05009385 C

### 4. Performance Analysis

In [4]:
# Combine all residuals
all_mae, all_rmse, all_fitting_score = [], [], []
positive_count = 0
total_residuals = 0

for result in results:
    residuals = result['residuals']
    all_mae.append(mean_absolute_error(result["actual"], result["predicted"]))
    all_rmse.append(np.sqrt(mean_squared_error(result["actual"], result["predicted"])))
    all_fitting_score.append(result["fitting_score"])

# Calculate statistics
avg_mae = np.mean(all_mae)
avg_rmse = np.mean(all_rmse)
ocs_list, dvr_list, sp_list = [], [], []

for score in all_fitting_score:
    ocs, dvr, sp = score
    ocs_list.append(ocs)
    dvr_list.append(dvr)
    sp_list.append(sp)
avg_fitting_score = (np.mean(ocs_list), np.mean(dvr_list), np.mean(sp_list))

print(f"\nAverage MAE: {avg_mae:.2f}")
print(f"Average RMSE: {avg_rmse:.2f}")
# print(f"Average fitting score: {avg_fitting_score:.2f}")
print(avg_fitting_score)


Average MAE: 4.13
Average RMSE: 4.91
(0.12426498314972576, 0.1397765064316779, 0.11761718745746058)


### 5. Visualisation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def plot_sarima_validation(result):
    """Plot training data, actual test values and predictions for a single ward"""
    plt.figure(figsize=(14, 7))

    # Extract data
    ward_code = result['ward_code']
    train_dates = burglary_monthly[burglary_monthly['Ward Code'] == ward_code]['Month'].iloc[:-26]
    train_counts = burglary_monthly[burglary_monthly['Ward Code'] == ward_code]['Count'].iloc[:-26]
    test_dates = burglary_monthly[burglary_monthly['Ward Code'] == ward_code]['Month'].iloc[-26:]

    # Create plots
    plt.plot(train_dates, train_counts, 'b-', label='Training Data')
    plt.plot(test_dates, result['actual'], 'ko-', label='Actual Test Values')
    plt.plot(test_dates, result['predicted'], 'r--', label='SARIMA Predictions')

    # Formatting
    plt.title(f'''SARIMA Validation for Ward {ward_code}
    Order: {result['model_order']}, Seasonal Order: {result['model_seasonal_order']}''')
    plt.xlabel('Date')
    plt.ylabel('Burglary Count')
    plt.legend()
    plt.grid(True)

    # Add metrics
    rmse = np.sqrt(mean_squared_error(result['actual'], result['predicted']))
    mae = mean_absolute_error(result['actual'], result['predicted'])
    plt.text(0.05, 0.95,
            f'Test RMSE: {rmse:.2f}\nTest MAE: {mae:.2f}',
            transform=plt.gca().transAxes,
            verticalalignment='top',
            bbox=dict(facecolor='white', alpha=0.8))

    plt.show()

# Plot first 3 wards as examples
for i, result in enumerate(results[:200]):
    plot_sarima_validation(result)
    print(f"\n{'-'*50}\nResidual Statistics for Ward {result['ward_code']}:")
    # print(f"Fitting Score: {result['fitting_score']:.2f}")
    print(result["fitting_score"])